In [ ]:
!pip install load_dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
from pathlib import Path
import re
import tensorflow as tf

from dotenv import load_dotenv
load_dotenv()

TRAIN_DATASET_PATH = "/content/drive/MyDrive/AWS/lamps/train"
VALIDATION_DATASET_PATH = "/content/drive/MyDrive/AWS/lamps/validation"
BATCH_SIZE = 32
N_CLASSES = 17
TOTAL_EPOCHS = 10

# Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=TRAIN_DATASET_PATH,
    target_size=[350, 350],
    class_mode='categorical'
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255
)

val_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=VALIDATION_DATASET_PATH,
    target_size=[350, 350],
    class_mode='categorical'
)

# Model
kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
trained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                      include_top=False,
                      weights='imagenet',
                      alpha=0.5,
                      input_shape=[350, 350, 3],
                      pooling='max')
output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=kernel_initializer)(trained_model.output)
model = tf.keras.Model(inputs=trained_model.input, outputs=output)

# Callback to save weights, based on val_acc, old version in keras
# Callback to save weights, based on val_accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  './checkpoints/{epoch:02d}_{val_accuracy:.4f}.h5',
  save_weights_only=False,
  verbose=1,
  monitor='val_accuracy',
  save_best_only=True,
  mode='max'
)


# Callbackto plot data on TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir='./logs/furniture_classifier',
  histogram_freq=0,
  batch_size=BATCH_SIZE
)

# Callback to reduce learning rate after plateaus
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor='val_acc',
  factor=0.5,
  patience=4,
  min_lr=1e-6
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_acc',
  patience=20,
  mode='max',
)

TRAIN_DATASET_SIZE = len(train_batches)
VAL_DATASET_SIZE   = len(val_batches)

# Weighted losses for class equilibrium
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), N_CLASSES*counts)))


if Path('./checkpoints/').exists():
  epoch_number_array = []
  val_accuracy_array = []
  file_name_array = []
  for file in os.listdir('./checkpoints/'):
    epoch, val_acc = re.search(r'(\d\d)_(\d\.\d{4})\.h5', file).group(1,2)
    epoch_number_array.append(int(epoch))
    val_accuracy_array.append(float(val_acc))
    file_name_array.append(file)

  if len(val_accuracy_array) == 0:
    INITIAL_EPOCH = 0
  else:
    highest_acc = val_accuracy_array.index(max(val_accuracy_array))
    INITIAL_EPOCH = epoch_number_array[highest_acc]
    model_checkpoint_callback.best = val_accuracy_array[highest_acc]
    model.load_weights('./checkpoints/'+file_name_array[highest_acc])
else:
  os.makedirs('./checkpoints/')
  INITIAL_EPOCH = 0


# Prepare model to run
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
              )

# Starts training the model
model.fit_generator(train_batches,
                    epochs=TOTAL_EPOCHS,
                    verbose=1,
                    steps_per_epoch=TRAIN_DATASET_SIZE,
                    validation_data=val_batches,
                    validation_steps=VAL_DATASET_SIZE,
                    initial_epoch=INITIAL_EPOCH,
                    class_weight=class_weigths,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, early_stopping_callback]
                    )

Found 850 images belonging to 17 classes.
Found 821 images belonging to 17 classes.


<ipython-input-4-a65ce018e0c6>:128: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batches,


Epoch 4/10
27/27 [==============================] - ETA: 0s - loss: 3.7698 - accuracy: 0.0541
Epoch 4: val_accuracy did not improve from 0.09710


27/27 [==============================] - 227s 8s/step - loss: 3.7698 - accuracy: 0.0541 - val_loss: 16.3220 - val_accuracy: 0.0609 - lr: 0.0100
Epoch 5/10
27/27 [==============================] - ETA: 0s - loss: 3.4831 - accuracy: 0.0612

In [ ]:
# Save the model
model.save("model_lamp.h5", save_format='h5', include_optimizer=True, save_traces=True)

# Load the saved model
loaded_model = tf.keras.models.load_model("model_lamp.h5")

# Test the model on a new image
image_path = "/content/drive/MyDrive/AWS/lamps/train/Asian/13898asian-table-lamps.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=[224, 224])
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create batch axis
img_array = img_array / 255.0 # Normalize pixel values
prediction = loaded_model.predict(img_array)
class_index = np.argmax(prediction)
class_names = list(train_batches.class_indices.keys())
print(class_index)
print(class_names)

predicted_class = class_names[class_index]
print(predicted_class)

1/1 [==============================] - 1s 649ms/step
16
['Asian', 'Beach', 'Contemporary', 'Craftsman', 'Eclectic', 'Farmhouse', 'Industrial', 'Mediterranean', 'Midcentury', 'Modern', 'Rustic', 'Scandinavian', 'Southwestern', 'Traditional', 'Transitional', 'Tropical', 'Victorian']
Victorian


In [ ]:
class_names = list(train_batches.class_indices.keys())
print(class_names)

['Asian', 'Beach', 'Contemporary', 'Craftsman', 'Eclectic', 'Farmhouse', 'Industrial', 'Mediterranean', 'Midcentury', 'Modern', 'Rustic', 'Scandinavian', 'Southwestern', 'Traditional', 'Transitional', 'Tropical', 'Victorian']


In [ ]:
import os
import numpy as np
from pathlib import Path
import re
import tensorflow as tf

from dotenv import load_dotenv
load_dotenv()

TRAIN_DATASET_PATH = "/content/drive/MyDrive/AWS/beds/train"
VALIDATION_DATASET_PATH = "/content/drive/MyDrive/AWS/beds/validation"
BATCH_SIZE = 32
N_CLASSES = 17
TOTAL_EPOCHS = 10

# Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=TRAIN_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255
)

val_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=VALIDATION_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

# Model
kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
trained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                      include_top=False,
                      weights='imagenet',
                      alpha=0.5,
                      input_shape=[224, 224, 3],
                      pooling='max')
output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=kernel_initializer)(trained_model.output)
model = tf.keras.Model(inputs=trained_model.input, outputs=output)

# Callback to save weights, based on val_acc, old version in keras
# Callback to save weights, based on val_accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  './checkpoints/{epoch:02d}_{val_accuracy:.4f}.h5',
  save_weights_only=False,
  verbose=1,
  monitor='val_accuracy',
  save_best_only=True,
  mode='max'
)


# Callbackto plot data on TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir='./logs/furniture_classifier',
  histogram_freq=0,
  batch_size=BATCH_SIZE
)

# Callback to reduce learning rate after plateaus
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor='val_acc',
  factor=0.5,
  patience=4,
  min_lr=1e-6
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_acc',
  patience=20,
  mode='max',
)

TRAIN_DATASET_SIZE = len(train_batches)
VAL_DATASET_SIZE   = len(val_batches)

# Weighted losses for class equilibrium
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), N_CLASSES*counts)))


if Path('./checkpoints/').exists():
  epoch_number_array = []
  val_accuracy_array = []
  file_name_array = []
  for file in os.listdir('./checkpoints/'):
    epoch, val_acc = re.search(r'(\d\d)_(\d\.\d{4})\.h5', file).group(1,2)
    epoch_number_array.append(int(epoch))
    val_accuracy_array.append(float(val_acc))
    file_name_array.append(file)

  if len(val_accuracy_array) == 0:
    INITIAL_EPOCH = 0
  else:
    highest_acc = val_accuracy_array.index(max(val_accuracy_array))
    INITIAL_EPOCH = epoch_number_array[highest_acc]
    model_checkpoint_callback.best = val_accuracy_array[highest_acc]
    model.load_weights('./checkpoints/'+file_name_array[highest_acc])
else:
  os.makedirs('./checkpoints/')
  INITIAL_EPOCH = 0


# Prepare model to run
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
              )

# Starts training the model
model.fit_generator(train_batches,
                    epochs=TOTAL_EPOCHS,
                    verbose=1,
                    steps_per_epoch=TRAIN_DATASET_SIZE,
                    validation_data=val_batches,
                    validation_steps=VAL_DATASET_SIZE,
                    initial_epoch=INITIAL_EPOCH,
                    class_weight=class_weigths,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, early_stopping_callback]
                    )

Found 723 images belonging to 17 classes.
Found 515 images belonging to 17 classes.


<ipython-input-1-8ff3f8db5089>:128: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batches,


Epoch 4/10
23/23 [==============================] - ETA: 0s - loss: 3.4603 - accuracy: 0.0415
Epoch 4: val_accuracy did not improve from 0.09710


23/23 [==============================] - 85s 3s/step - loss: 3.4603 - accuracy: 0.0415 - val_loss: 20.7989 - val_accuracy: 0.0291 - lr: 0.0100
Epoch 5/10
23/23 [==============================] - ETA: 0s - loss: 7.0458 - accuracy: 0.0553
Epoch 5: val_accuracy did not improve from 0.09710


23/23 [==============================] - 74s 3s/step - loss: 7.0458 - accuracy: 0.0553 - val_loss: 20.8759 - val_accuracy: 0.0136 - lr: 0.0100
Epoch 6/10
23/23 [==============================] - ETA: 0s - loss: 3.5873 - accuracy: 0.0443
Epoch 6: val_accuracy did not improve from 0.09710


23/23 [==============================] - 72s 3s/step - loss: 3.5873 - accuracy: 0.0443 - val_loss: 15.4380 - val_accuracy: 0.0136 - lr: 0.0100
Epoch 7/10
23/23 [==============================] - ETA: 0s - loss: 3.5208 - accuracy: 0.0443
Epoch 7: val_accuracy did not improve from 0.09710


23/23 [==============================] - 73s 3s/step - loss: 3.5208 - accuracy: 0.0443 - val_loss: 11.4960 - val_accuracy: 0.0971 - lr: 0.0100
Epoch 8/10
23/23 [==============================] - ETA: 0s - loss: 3.2095 - accuracy: 0.0553
Epoch 8: val_accuracy did not improve from 0.09710


23/23 [==============================] - 72s 3s/step - loss: 3.2095 - accuracy: 0.0553 - val_loss: 19.1008 - val_accuracy: 0.0311 - lr: 0.0100
Epoch 9/10
23/23 [==============================] - ETA: 0s - loss: 3.2763 - accuracy: 0.0512
Epoch 9: val_accuracy did not improve from 0.09710


23/23 [==============================] - 76s 3s/step - loss: 3.2763 - accuracy: 0.0512 - val_loss: 9.0876 - val_accuracy: 0.0311 - lr: 0.0100
Epoch 10/10
23/23 [==============================] - ETA: 0s - loss: 2.9805 - accuracy: 0.0650
Epoch 10: val_accuracy did not improve from 0.09710


23/23 [==============================] - 73s 3s/step - loss: 2.9805 - accuracy: 0.0650 - val_loss: 8.2814 - val_accuracy: 0.0311 - lr: 0.0100


In [ ]:
# Save the model
model.save("model_bed.h5", save_format='h5', include_optimizer=True, save_traces=True)

# Load the saved model
loaded_model = tf.keras.models.load_model("model_bed.h5")

# Test the model on a new image
image_path = "/content/drive/MyDrive/AWS/lamps/train/Asian/13898asian-table-lamps.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=[224, 224])
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create batch axis
img_array = img_array / 255.0 # Normalize pixel values
prediction = loaded_model.predict(img_array)
class_index = np.argmax(prediction)
class_names = list(train_batches.class_indices.keys())
print(class_index)
predicted_class = class_names[class_index]
print(predicted_class)

1/1 [==============================] - 1s 622ms/step
0
Asian


In [ ]:
import os
import numpy as np
from pathlib import Path
import re
import tensorflow as tf

from dotenv import load_dotenv
load_dotenv()

TRAIN_DATASET_PATH = "/content/drive/MyDrive/AWS/dressers/train"
VALIDATION_DATASET_PATH = "/content/drive/MyDrive/AWS/dressers/validation"
BATCH_SIZE = 32
N_CLASSES = 17
TOTAL_EPOCHS = 10

# Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=TRAIN_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255
)

val_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=VALIDATION_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

# Model
kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
trained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                      include_top=False,
                      weights='imagenet',
                      alpha=0.5,
                      input_shape=[224, 224, 3],
                      pooling='max')
output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=kernel_initializer)(trained_model.output)
model = tf.keras.Model(inputs=trained_model.input, outputs=output)

# Callback to save weights, based on val_acc, old version in keras
# Callback to save weights, based on val_accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  './checkpoints/{epoch:02d}_{val_accuracy:.4f}.h5',
  save_weights_only=False,
  verbose=1,
  monitor='val_accuracy',
  save_best_only=True,
  mode='max'
)


# Callbackto plot data on TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir='./logs/furniture_classifier',
  histogram_freq=0,
  batch_size=BATCH_SIZE
)

# Callback to reduce learning rate after plateaus
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor='val_acc',
  factor=0.5,
  patience=4,
  min_lr=1e-6
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_acc',
  patience=20,
  mode='max',
)

TRAIN_DATASET_SIZE = len(train_batches)
VAL_DATASET_SIZE   = len(val_batches)

# Weighted losses for class equilibrium
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), N_CLASSES*counts)))


if Path('./checkpoints/').exists():
  epoch_number_array = []
  val_accuracy_array = []
  file_name_array = []
  for file in os.listdir('./checkpoints/'):
    epoch, val_acc = re.search(r'(\d\d)_(\d\.\d{4})\.h5', file).group(1,2)
    epoch_number_array.append(int(epoch))
    val_accuracy_array.append(float(val_acc))
    file_name_array.append(file)

  if len(val_accuracy_array) == 0:
    INITIAL_EPOCH = 0
  else:
    highest_acc = val_accuracy_array.index(max(val_accuracy_array))
    INITIAL_EPOCH = epoch_number_array[highest_acc]
    model_checkpoint_callback.best = val_accuracy_array[highest_acc]
    model.load_weights('./checkpoints/'+file_name_array[highest_acc])
else:
  os.makedirs('./checkpoints/')
  INITIAL_EPOCH = 0


# Prepare model to run
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
              )

# Starts training the model
model.fit_generator(train_batches,
                    epochs=TOTAL_EPOCHS,
                    verbose=1,
                    steps_per_epoch=TRAIN_DATASET_SIZE,
                    validation_data=val_batches,
                    validation_steps=VAL_DATASET_SIZE,
                    initial_epoch=INITIAL_EPOCH,
                    class_weight=class_weigths,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, early_stopping_callback]
                    )

Found 741 images belonging to 17 classes.
Found 655 images belonging to 17 classes.


<ipython-input-1-d6cef41cc860>:128: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batches,


Epoch 4/10
24/24 [==============================] - ETA: 0s - loss: 4.8999 - accuracy: 0.0688
Epoch 4: val_accuracy did not improve from 0.09710


24/24 [==============================] - 85s 3s/step - loss: 4.8999 - accuracy: 0.0688 - val_loss: 20.3091 - val_accuracy: 0.0595 - lr: 0.0100
Epoch 5/10
24/24 [==============================] - ETA: 0s - loss: 4.4646 - accuracy: 0.0729
Epoch 5: val_accuracy did not improve from 0.09710


24/24 [==============================] - 75s 3s/step - loss: 4.4646 - accuracy: 0.0729 - val_loss: 16.4642 - val_accuracy: 0.0702 - lr: 0.0100
Epoch 6/10
24/24 [==============================] - ETA: 0s - loss: 3.8390 - accuracy: 0.0486
Epoch 6: val_accuracy did not improve from 0.09710


24/24 [==============================] - 78s 3s/step - loss: 3.8390 - accuracy: 0.0486 - val_loss: 13.6241 - val_accuracy: 0.0061 - lr: 0.0100
Epoch 7/10
24/24 [==============================] - ETA: 0s - loss: 3.4718 - accuracy: 0.0364
Epoch 7: val_accuracy did not improve from 0.09710


24/24 [==============================] - 78s 3s/step - loss: 3.4718 - accuracy: 0.0364 - val_loss: 13.8935 - val_accuracy: 0.0595 - lr: 0.0100
Epoch 8/10
24/24 [==============================] - ETA: 0s - loss: 3.0624 - accuracy: 0.0486
Epoch 8: val_accuracy did not improve from 0.09710


24/24 [==============================] - 74s 3s/step - loss: 3.0624 - accuracy: 0.0486 - val_loss: 14.6161 - val_accuracy: 0.0763 - lr: 0.0100
Epoch 9/10
24/24 [==============================] - ETA: 0s - loss: 2.9787 - accuracy: 0.0499
Epoch 9: val_accuracy did not improve from 0.09710


24/24 [==============================] - 74s 3s/step - loss: 2.9787 - accuracy: 0.0499 - val_loss: 14.4617 - val_accuracy: 0.0595 - lr: 0.0100
Epoch 10/10
24/24 [==============================] - ETA: 0s - loss: 3.0397 - accuracy: 0.0432
Epoch 10: val_accuracy did not improve from 0.09710


24/24 [==============================] - 75s 3s/step - loss: 3.0397 - accuracy: 0.0432 - val_loss: 11.1534 - val_accuracy: 0.0763 - lr: 0.0100


In [ ]:
# Save the model
model.save("model_dresser.h5", save_format='h5', include_optimizer=True, save_traces=True)

# Load the saved model
loaded_model = tf.keras.models.load_model("model_bed.h5")

# Test the model on a new image
image_path = "/content/drive/MyDrive/AWS/lamps/train/Asian/13898asian-table-lamps.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=[224, 224])
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create batch axis
img_array = img_array / 255.0 # Normalize pixel values
prediction = loaded_model.predict(img_array)
class_index = np.argmax(prediction)
class_names = list(train_batches.class_indices.keys())
print(class_index)
predicted_class = class_names[class_index]
print(predicted_class)
print(class_names)

1/1 [==============================] - 1s 603ms/step
0
Asian
['Asian', 'Beach', 'Contemporary', 'Craftsman', 'Eclectic', 'Farmhouse', 'Industrial', 'Mediterranean', 'Midcentury', 'Modern', 'Rustic', 'Scandinavian', 'Southwestern', 'Traditional', 'Transitional', 'Tropical', 'Victorian']


In [ ]:
import os
import numpy as np
from pathlib import Path
import re
import tensorflow as tf

from dotenv import load_dotenv
load_dotenv()

TRAIN_DATASET_PATH = "/content/drive/MyDrive/AWS/chairs/train"
VALIDATION_DATASET_PATH = "/content/drive/MyDrive/AWS/chairs/validation"
BATCH_SIZE = 32
N_CLASSES = 17
TOTAL_EPOCHS = 10

# Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=TRAIN_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255
)

val_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=VALIDATION_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

# Model
kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
trained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                      include_top=False,
                      weights='imagenet',
                      alpha=0.5,
                      input_shape=[224, 224, 3],
                      pooling='max')
output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=kernel_initializer)(trained_model.output)
model = tf.keras.Model(inputs=trained_model.input, outputs=output)

# Callback to save weights, based on val_acc, old version in keras
# Callback to save weights, based on val_accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  './checkpoints/{epoch:02d}_{val_accuracy:.4f}.h5',
  save_weights_only=False,
  verbose=1,
  monitor='val_accuracy',
  save_best_only=True,
  mode='max'
)


# Callbackto plot data on TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir='./logs/furniture_classifier',
  histogram_freq=0,
  batch_size=BATCH_SIZE
)

# Callback to reduce learning rate after plateaus
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor='val_acc',
  factor=0.5,
  patience=4,
  min_lr=1e-6
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_acc',
  patience=20,
  mode='max',
)

TRAIN_DATASET_SIZE = len(train_batches)
VAL_DATASET_SIZE   = len(val_batches)

# Weighted losses for class equilibrium
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), N_CLASSES*counts)))


if Path('./checkpoints/').exists():
  epoch_number_array = []
  val_accuracy_array = []
  file_name_array = []
  for file in os.listdir('./checkpoints/'):
    epoch, val_acc = re.search(r'(\d\d)_(\d\.\d{4})\.h5', file).group(1,2)
    epoch_number_array.append(int(epoch))
    val_accuracy_array.append(float(val_acc))
    file_name_array.append(file)

  if len(val_accuracy_array) == 0:
    INITIAL_EPOCH = 0
  else:
    highest_acc = val_accuracy_array.index(max(val_accuracy_array))
    INITIAL_EPOCH = epoch_number_array[highest_acc]
    model_checkpoint_callback.best = val_accuracy_array[highest_acc]
    model.load_weights('./checkpoints/'+file_name_array[highest_acc])
else:
  os.makedirs('./checkpoints/')
  INITIAL_EPOCH = 0


# Prepare model to run
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
              )

# Starts training the model
model.fit_generator(train_batches,
                    epochs=TOTAL_EPOCHS,
                    verbose=1,
                    steps_per_epoch=TRAIN_DATASET_SIZE,
                    validation_data=val_batches,
                    validation_steps=VAL_DATASET_SIZE,
                    initial_epoch=INITIAL_EPOCH,
                    class_weight=class_weigths,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, early_stopping_callback]
                    )

Found 850 images belonging to 17 classes.
Found 829 images belonging to 17 classes.
3201480/3201480 [==============================] - 0s 0us/step


<ipython-input-4-50e098897b78>:128: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batches,


Epoch 1/10
27/27 [==============================] - ETA: 0s - loss: 8.2127 - accuracy: 0.0635
Epoch 1: val_accuracy improved from -inf to 0.06031, saving model to ./checkpoints/01_0.0603.h5


27/27 [==============================] - 203s 7s/step - loss: 8.2127 - accuracy: 0.0635 - val_loss: 17.7096 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 2/10
27/27 [==============================] - ETA: 0s - loss: 4.6110 - accuracy: 0.0529
Epoch 2: val_accuracy did not improve from 0.06031


27/27 [==============================] - 135s 5s/step - loss: 4.6110 - accuracy: 0.0529 - val_loss: 19.8182 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 3/10
27/27 [==============================] - ETA: 0s - loss: 3.9845 - accuracy: 0.0600
Epoch 3: val_accuracy did not improve from 0.06031


...

27/27 [==============================] - 148s 5s/step - loss: 4.3997 - accuracy: 0.0671 - val_loss: 55.3545 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 7/10
27/27 [==============================] - ETA: 0s - loss: 7.3674 - accuracy: 0.0671
Epoch 7: val_accuracy did not improve from 0.06031


27/27 [==============================] - 139s 5s/step - loss: 7.3674 - accuracy: 0.0671 - val_loss: 23.3176 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 8/10
27/27 [==============================] - ETA: 0s - loss: 4.1260 - accuracy: 0.0682
Epoch 8: val_accuracy did not improve from 0.06031


27/27 [==============================] - 153s 5s/step - loss: 4.1260 - accuracy: 0.0682 - val_loss: 13.8145 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 9/10
27/27 [==============================] - ETA: 0s - loss: 3.7523 - accuracy: 0.0800
Epoch 9: val_accuracy did not improve from 0.06031


27/27 [==============================] - 145s 5s/step - loss: 3.7523 - accuracy: 0.0800 - val_loss: 11.7679 - val_accuracy: 0.0603 - lr: 0.0100
Epoch 10/10
27/27 [==============================] - ETA: 0s - loss: 3.4823 - accuracy: 0.0529
Epoch 10: val_accuracy did not improve from 0.06031


27/27 [==============================] - 147s 5s/step - loss: 3.4823 - accuracy: 0.0529 - val_loss: 18.2067 - val_accuracy: 0.0350 - lr: 0.0100


In [ ]:
# Save the model
model.save("model_chair.h5", save_format='h5', include_optimizer=True, save_traces=True)

# Load the saved model
loaded_model = tf.keras.models.load_model("model_chair.h5")

# Test the model on a new image
image_path = "/content/drive/MyDrive/AWS/lamps/train/Asian/13898asian-table-lamps.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=[224, 224])
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create batch axis
img_array = img_array / 255.0 # Normalize pixel values
prediction = loaded_model.predict(img_array)
class_index = np.argmax(prediction)
class_names = list(train_batches.class_indices.keys())
print(class_index)
predicted_class = class_names[class_index]
print(predicted_class)
print(class_names)

1/1 [==============================] - 1s 1s/step
12
Southwestern
['Asian', 'Beach', 'Contemporary', 'Craftsman', 'Eclectic', 'Farmhouse', 'Industrial', 'Mediterranean', 'Midcentury', 'Modern', 'Rustic', 'Scandinavian', 'Southwestern', 'Traditional', 'Transitional', 'Tropical', 'Victorian']
